[this doc on github](https://github.com/dotnet/interactive/tree/main/samples/notebooks/fsharp/Samples)

### Referencing the package


First, get the package and open the namespaces:

In [ ]:
#r "nuget:Microsoft.Data.Analysis, 0.21.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: Plotly.Net, 4.2.0"

open Microsoft.Data.Analysis

### Creating a data frame

Create 3 columns to hold values of types `DateTime`, `int`, and `string`

In [ ]:
let dateTimes = DateTimeDataFrameColumn("DateTimes") // Default length is 0.
let ints = Int32DataFrameColumn("Ints", 6L) // Makes a column of Length 3. Filles with nulls initially.
let strings = StringDataFrameColumn("Strings", 6L)

Add some datetimes

In [ ]:
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/01"));
dateTimes.Append(DateTime.Parse("2019/01/02"));
dateTimes.Append(DateTime.Parse("2019/02/02"));
dateTimes.Append(DateTime.Parse("2019/02/02"));
dateTimes.Append(DateTime.Parse("2019/03/02"));

Create a `DataFrame` with 3 columns

In [ ]:
let df = DataFrame([dateTimes; ints; strings]: DataFrameColumn list)

### Adding better default formatting for data frames

Create a formatter for data frames and data frame rows.

In [ ]:
module DateFrameFormatter = 
    
    // Locally open the F# HTML DSL.
    open Html

    let maxRows = 20

    Formatter.Register<DataFrame>((fun (df: DataFrame) (writer: TextWriter) ->

        let take = 20
        table [] [
          thead [] [
            th [] [ str "Index" ]
            for c in df.Columns do
              th [] [ str c.Name]
          ]
          tbody [] [
            for i in 0 .. min maxRows (int df.Rows.Count - 1) do
              tr [] [
                td [] [ i ]
                for o in df.Rows.[int64 i] do
                  td [] [ o ]
              ]
          ]
        ]
        |> writer.Write

    ), mimeType = "text/html")
    
    Formatter.Register<DataFrameRow>((fun (row: DataFrameRow) (writer: TextWriter) ->

        table [] [
          tbody [] [
            tr [] [
              for o in row do
                td [] [ o ] 
            ]
          ]
        ]
        |> writer.Write

    ), mimeType = "text/html")
    

Now view the data frame: 

In [ ]:
df

### Modifying data frames

Change a value directly through df:

In [ ]:
df.[0L, 1] <- 10
df

We can also modify the values in the columns through indexers defined in `PrimitiveDataColumn` and `StringColumn`

In [ ]:
ints.[1L] <- Nullable 100
strings.[1L] <- "Foo!"
df

Check the data type

In [ ]:
df.Info()

The `DataFrame` and the base `DataFrameColumn` class that all columns derive from expose a number of useful APIs: binary operations, computations, joins, merges, handling missing values and more.

In [ ]:
df.["Ints"].Add(5, inPlace=true)
df

In [ ]:
df.["Ints"] <- (ints / 5) * 20
df

Let's `null` it up!

In [ ]:
df.["Ints"].FillNulls(100, inPlace=true)
df.["Strings"].FillNulls("Bar", inPlace=true)
df

DataFrame exposes `Columns` property that we can enumerate over to access our columns. Here's how you can access the first row, though.

In [ ]:
let row0 = df.Rows.[0L]
row0

In [ ]:
row0

### Filtering and sorting data frames

Let's take a look at `Filter`, `Sort`, and `GroupBy`.

In [ ]:
// Sort our dataframe using the Ints column
df.OrderBy("Ints")

In [ ]:
//Clone dataframe
let newDf = df.Clone();

//Add new column
let newColumn = new StringDataFrameColumn("Month", (df["DateTimes"] :?> DateTimeDataFrameColumn |> Seq.map (fun x -> x.Value.ToString "MMMM")) |> Seq.toArray);
newDf.Columns.Add(newColumn);

In [ ]:
// GroupBy
let grouped = newDf.GroupBy("Month")

//Show grouped data
let groupedDf = grouped.Head(10);
groupedDf

In [ ]:
let intGroupSum = grouped.Sum("Ints");
intGroupSum

### Charting columns from data frames

In [ ]:
open Plotly.NET
open System.Linq

In [ ]:
display(Chart2D.Chart.Column<int, string, string, string, string>((intGroupSum["Ints"] :?> Int32DataFrameColumn |> Seq.map (fun x -> if x = Nullable() then 0 else x.Value)), intGroupSum["Month"] :?> StringDataFrameColumn ))